# Classificatie van woorden in het Excelsheet

In dit Jupyter Notebook lezen we de genoemde woorden uit in elk artikel en slaan we ze op in een SQLite-database.

In [1]:
import pandas as pd
import numpy  as np

EXCEL_BESTAND  = 'preprocessed_data.xlsx'
SQLITE_BESTAND = 'ftm.db'     # Het bestand mag elke naam hebben, maar iets wat eindigt op `.db`, is aanbevolen.

We beginnen met het lezen van het bestand. Het bestand wordt als workbook in het geheugen geladen, waarna we het overzetten naar een bestand.

In [2]:
preprocessed_data = pd.read_excel(EXCEL_BESTAND)
preprocessed_data

,Unnamed: 0,id,type,date,betterDate,title,abstract
0,18,13.16,Overig,2015,NaT,Verzoek_regulier__facultatief_advies_uitgebr_p...,\n\n\n\n\n13.16 \n \n\nOns briefkenmerk: ...
1,29,8.17,Correspondentie,2007-2013,NaT,Toezicht_ Correspondentie n.a.v. Toezicht en H...,\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
2,31,22.11,Mail,??,NaT,FW Draaiboek demonstratie NAM AZC 24 sept...,"\n\n\nDate : 1-1-0001 00:00:00 \nTo : ""Marco O..."
3,32,32.3,Correspondentie,NaN,NaT,Uitgaande mail mededeling akkoord bijdrage aan...,\n\n\n\n\n| bijdrage gemeente Assen Pagina 1 \...
4,35,8.10,Vergunning,1997,NaT,Rapportage n.a.v. vergunningen-meldingen.pdf_j...,\n\n\n\n\n \ncle) \n \n\n\n\n\n\n\n\n\n\n\n...
...,...,...,...,...,...,...,...
2504,2537,32.1,Bestuurlijk besluit,2005-04-26 00:00:00,NaT,BenW besluit BB54473 inzake deelname gem Assen...,\n\n\nÀ \n\nfifi Gemeente Assen \n\n\n\n \n...
2505,2538,32.5,Document,2005-04-01 00:00:00,NaT,Persbericht Energy Valley_,\n\n\n\n\n32.5 \n \n\nÓ Gemeente Assen \n...
2506,2539,3.35,Mail,2004-12-13 00:00:00,2016-09-09 10:47:53,Melding betoging 24 september NAM,\n\n\n\n\n\n\nDate : 9-9-2016 10:47:53 \nF B n...
2507,2540,3.40,Document,2004-12-13 00:00:00,NaT,Locaties in Groningen_,\n\n\nCentrale meldkamer NAM Assen (CMK) \n\nB...


We passen toe dat elk woord in de string wordt opgeslagen in een dictionary.

In [3]:
CLASSIFIED_WORDS = {}

def add_words_to_dictionary(value):
    value = str(value).replace('\t', '    ').replace('\n', ' ').replace('\r', ' ').lower().split(' ')
    
    for word in value:
        if word == '':
            continue
        CLASSIFIED_WORDS[word] = 1 if word not in CLASSIFIED_WORDS else CLASSIFIED_WORDS[word] + 1

preprocessed_data['title']   .apply(add_words_to_dictionary)
preprocessed_data['abstract'].apply(add_words_to_dictionary)

CLASSIFIED_WORDS

{'verzoek_regulier__facultatief_advies_uitgebr_proc.doc_dd.': 1,
 'toezicht_': 2,
 'correspondentie': 72,
 'n.a.v.': 164,
 'toezicht': 90,
 'en': 70931,
 'handhaving.pdf_': 2,
 'fw': 360,
 'draaiboek': 110,
 'demonstratie': 75,
 'nam': 11013,
 'azc': 33,
 '24': 1061,
 'september': 1037,
 '2016': 3123,
 '??.pdf': 1,
 'uitgaande': 104,
 'mail': 857,
 'mededeling': 81,
 'akkoord': 305,
 'bijdrage': 461,
 'aan': 20841,
 'st._ongedateerd.pdf': 1,
 'rapportage': 312,
 'vergunningen-meldingen.pdf_juli-dec': 1,
 'diverse': 454,
 'tekeningen.pdf_dd': 1,
 'inkomende': 6,
 '1.pdf_dd': 1,
 'toezicht_rapporteren': 2,
 'vergunning_correspondentie': 3,
 'vergunningen-meldingen.pdf_': 2,
 'diversen_diverse': 1,
 'correspondentie.pdf_': 1,
 'handhaving': 109,
 '1.pdf_': 6,
 '1324187_1404734980943_3b._offerteformulier_inhuur.xls_?.pdf': 1,
 '1674527_1432733943438_3b._offerteformulier_inhuur.xls_?.pdf': 1,
 'brochure': 53,
 'drenthe': 2737,
 '4.0': 284,
 'met': 25123,
 'kaft.pdf_apr.mei': 1,
 '320755_132

In [4]:
CLASSIFIED_WORDS['mailcontact']

5

In [5]:
from collections import Counter

def information_rate(content):
    words = str(content).replace('\t', '    ').replace('\n', ' ').replace('\r', ' ').lower().split(' ')
    total = 0
    
    words = Counter(words)
    
    for word in words:
        if word == '':
            continue
            
        rarity     = CLASSIFIED_WORDS[word]
        appearance = words[word]
    
        total += appearance * appearance / rarity

    return total

information_rate('mailcontact')

0.2

We calculate the word relevance and then use the sigmoid function to spread it nicely between 0 and 1.

In [12]:
preprocessed_data['info_score']  = preprocessed_data['title']   .apply(information_rate)
preprocessed_data['info_score'] += preprocessed_data['abstract'].apply(information_rate)

preprocessed_data['norm_info_score'] = 1 / (1 + np.exp(-1 * np.log10(preprocessed_data['info_score'])))

preprocessed_data.sort_values(by='norm_info_score')

,Unnamed: 0,id,type,date,betterDate,title,abstract,info_score,norm_info_score
1602,1635,16.54,Mail,2016-12-13 00:00:00,NaT,2e versie advies winningsplan,\n\n\nRapport \nDecember 2016 \n \n\n1e conce...,0.025085,0.167900
1145,1178,16.52,Mail,2017-05-12 00:00:00,NaT,1e concept zienswijze,\n\n\n11 Mei 2017 \n \nconcept \n\nZienswijze...,0.431546,0.409756
592,625,5.8,Rapport,2018-04-30 00:00:00,NaT,2018-11574 Taxatierapport Schepersmaat Assen.pdf_,\n\n\nTaxatierapport \n\n \n\n\n\n\n\n\n\n\n\n...,1.451706,0.540382
1800,1833,16.1324,Mail,2016-11-09 00:00:00,2016-11-09 12:47:56,raadvoorstel adviesrecht gaswinningsplan Weste...,\n\n\nDate : 9-11-2016 12:47:56 \nF nen! \nTo ...,2.370067,0.592609
1771,1804,16.153,Mail,2016-11-19 00:00:00,2016-11-29 14:53:38,bestuurlijke afstemming advies winningsplan,\n\n\nDate : 29-11-2016 14:53:38 \n\nFrom : 16...,2.542843,0.599965
...,...,...,...,...,...,...,...,...,...
1,29,8.17,Correspondentie,2007-2013,NaT,Toezicht_ Correspondentie n.a.v. Toezicht en H...,\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,1331.444640,0.957885
616,649,31.116,Mail,2018-04-13 00:00:00,NaT,Interview met XXXXXXXXXX in NRC,\n\n\n\n\n\n\n31.116 \n \n \n\nDate 1342018 ...,1371.491236,0.958401
2369,2402,12.22,Document,2011-08-02 00:00:00,NaT,20110801 LPG Berekening Shell BP Stadsbedri...,\n\n\n\n\n \nLPG groepsrisico berekeningsmo...,1421.785317,0.959020
141,174,21.43,Mail,2019-02-06 00:00:00,2019-02-06 14:58:57,productiehoeveelheden gaswinning uit kleine...,\n\n\n\n\n21.43 \n Date : 6-2-2019 14:58:57...,2024.851352,0.964647


In [7]:
print(preprocessed_data.loc[2334]['abstract'])




Gemeente Assen 

Beleid 

Datum: 06-07-2012 
Eenheitsmanager/ J 

Progr. Manager: ú 



Telefoan: 
Portef. HI k LE 

Registr.nr,: LE oonso) P- ja ar 

Instemming verkregen en Kopie gezonden aan: 

Datum Eenheid Program 



FJ Dijkstra: 
1e 



enbare besluiten WJa INee 
Bekendmakingsplicht: Ja ZNee 
or. Oda SlNee 

Bij besluit in mandaat: 

Mardaatnummer- 

Informeren van de re 5 

Waa hetef 

Vervolgprocedure: 
Behandeling in de raad: 

D Informatief 
D Opiniërend 
D Bestuitvormend 



Afhandelen door 

Kopie besluit 5 

Depaneren Paraaf 

ha 1659254 



32.18 
      

Voorstel aan B&W 

  
Onderwerp: 

Financiering Energy Valley 4 (2012-2015) 



Voorstel: 

Ll. Instemmen met het strategieplan Energy Valley 4 
(2012-2015) en het actieplan voor 2012, 

2. Akkoord gaan met het (mede) financieren van 

Energy Valley 4 voor een totaal van €200,00 
voor £ jaar, in 2 termijnen. 

3. Akkoord gean met bijgevoegde raadsbrief vin de 
raad te informeren over it onderwerp. 

Achtergrond: , = 

In [8]:
len(CLASSIFIED_WORDS)

101911

In [9]:
np.arange(len(CLASSIFIED_WORDS))

array([     0,      1,      2, ..., 101908, 101909, 101910])

In [15]:
common_words = [w for w in CLASSIFIED_WORDS.keys()]
common_words.sort(key=lambda x : CLASSIFIED_WORDS[x], reverse=True)
common_words[0:15]

['de',
 'van',
 'het',
 'en',
 'in',
 'een',
 'is',
 'op',
 'voor',
 'te',
 'dat',
 'met',
 'aan',
 'winningsplan',
 'u']

## Woorden zoeken & matchen

Tot nu toe hebben we de informatiedichtheid van artikelen bepaald. In dit stuk gaan we kijken hoe we artikelen kunnen vinden die de opgegeven zoekwoorden zo veel mogelijk matchen.

1.0